In [ ]:
import math
import pandas as pd
import numpy as np
import datetime as dt
import yfinance as yf
import matplotlib.pyplot as plt
import seaborn as sn

stocks = ['AAPL', 'TSLA', 'NVDA', 'K', 'CRSP', 'NEM', 'PLTR']

end_date = dt.datetime.today()
start_date = end_date - dt.timedelta(2*365)

price = yf.download(stocks, period = '2y')['Adj Close'].pct_change().dropna()

returns = price

class EfficientFrontier:
    def __init__(self,returns, assets):
        self.returns = returns
        self.assets = assets
 
    def Calc(self):
        self.tbl = pd.DataFrame(index = self.assets)
        self.tbl['Mean'] = self.returns.mean()
        self.tbl['Variance'] = self.returns.var()
        return self.tbl.T
   
    def correlation(self):
        self.tbl = pd.DataFrame(index = self.assets)
        self.corr = self.returns.corr()
        return self.corr
    
    #code taken from https://www.youtube.com/watch?v=Isutk-wqJfE
    def PortfolioReturn(self, weight):
        return np.sum(self.returns.mean() * weight) * 252
    
    #code taken from https://www.youtube.com/watch?v=Isutk-wqJfE
    def PortfolioVolatility(self, weight):
        return np.sqrt(np.dot(weight.T, np.dot(self.returns.cov() * 252, weight))) 
returnCol = []
volatilityCol = []
weightsCol = []
sharperatioCol = []

ef = EfficientFrontier(returns = returns, assets = stocks)
    
for i in range (1000):
    
    weights = np.random.random(len(stocks))
    weights /= np.sum(weights)
    weightsCol.append(weights)
    port_return = ef.PortfolioReturn(weights)
    port_volatility = ef.PortfolioVolatility(weights)
    returnCol.append(port_return)
    volatilityCol.append(port_volatility)
    
ER = np.array(returnCol)
EV = np.array(volatilityCol)
sharpe_ratio = (ER - 0.01)/ EV

display(ef.Calc().T, ef.correlation())

plt.figure(figsize=(6, 6))
sn.heatmap(returns.corr(), annot=True, cmap="coolwarm")

plt.figure(figsize = (10, 6))
plt.scatter(EV, ER, c = sharpe_ratio, cmap = 'viridis')
plt.title('Efficient Frontier')
plt.xlabel('expected volatility')
plt.ylabel('expected return')
plt.colorbar(label = 'Sharpe ratio')
plt.show()


[*********************100%***********************]  7 of 7 completed
